In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


In [0]:
!ls

sample_data


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()                                                                                                           
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive



In [0]:
!ls

drive  sample_data


In [0]:
cp -r drive/My\ Drive/caps final/

In [0]:
cp -r drive/My\ Drive/stanford_data/vgg_feats.npy final/

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import io

batch_size = 200 # Batch size for training.
epochs = 10
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = 40000 # Number of samples to train on.

In [0]:
# from __future__ import print_function

# from keras.models import Model
# from keras.layers import Input, LSTM, Dense
# import numpy as np
# import io

# batch_size = 50 # Batch size for training.
# epochs = 200
# latent_dim = 512  # Latent dimensionality of the encoding space.
# num_samples = 500 # Number of samples to train on.

/content


In [0]:
data_path = 'final/wholeCaptionSet.txt'

In [0]:
# cp -r drive/wholedata/ final

In [0]:
img_mat_old=np.load('final/6k_train_ims.npy')   

In [0]:
img_mat_old=np.load('drive/My Drive/f8ktest/f8k_test_ims.npy')   

In [0]:
img_mat_old[0]/2

array([0.0070044, 0.       , 0.       , ..., 0.       , 0.       ,
       0.       ], dtype=float32)

In [0]:

img_mat_new[0]

array([ 0.70528084, -0.        ,  0.5142621 , ..., -0.        ,
       -0.        , -0.        ], dtype=float32)

In [0]:
img_mat_new=np.load('final/vgg_feats.npy')   

In [0]:
with io.open(data_path, 'r', encoding='utf-8') as f:
    lines_old = f.read().split('\n')
print(img_mat_old.shape)
print(len(lines_old)) 

img_mat=img_mat_old
lines=lines_old
img_mat = img_mat*10
#img_mat=img_mat_old
#lines=lines_old
#img_mat=10*img_mat

(5000, 4096)
40001


In [0]:
len(lines)

6001

In [0]:
img_mat[0]

array([0.       , 0.       , 0.       , ..., 0.       , 0.5696953,
       0.       ], dtype=float32)

In [0]:
img_mat[1]

array([0.       , 0.       , 0.       , ..., 0.       , 0.5696953,
       0.       ], dtype=float32)

In [0]:
img_mat[5]

array([0.29436243, 0.        , 0.        , ..., 0.11582039, 0.        ,
       0.        ], dtype=float32)

In [0]:
input_words = set()
target_words = set()
input_texts = []
target_texts = []
k=0
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text = line
    target_text = line
    #print(input_text)
    #print(target_text)
    #print(img_mat[k])
    target_text = 'STR ' + target_text + ' END'
    inp=input_text.split()
    #inp=input_text.split(' ')
    tar=target_text.split()
    #tar=target_text.split(' ')

    input_texts.append(inp)
    target_texts.append(tar)
    for wrd in inp:
            input_words.add(wrd)
    for wrd in tar:
            target_words.add(wrd)
    k=k+1
            
#print(input_words)
#print(target_words)
#print(input_texts)
#print(target_texts)
input_words.add('STR')
input_words.add('END')

In [0]:
input_texts[0]

[u'A',
 u'black',
 u'dog',
 u'is',
 u'running',
 u'after',
 u'a',
 u'white',
 u'dog',
 u'in',
 u'the',
 u'snow',
 u'.']

In [0]:
target_texts[0]

[u'STR',
 u'A',
 u'black',
 u'dog',
 u'is',
 u'running',
 u'after',
 u'a',
 u'white',
 u'dog',
 u'in',
 u'the',
 u'snow',
 u'.',
 u'END']

In [0]:
input_words = sorted(list(input_words ))
target_words  = sorted(list(target_words ))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])+2
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 40000
Number of unique input tokens: 9486
Number of unique output tokens: 9486
Max sequence length for inputs: 40
Max sequence length for outputs: 40


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_words )])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_words )])
print(input_token_index)
print(target_token_index)


{u'raining': 6837, u'Gymnast': 478, u'foul': 4053, u'griding': 4356, u'four': 4057, u'neckties': 5814, u'woods': 9382, u'Olympics': 731, u'hanging': 4487, u'woody': 9383, u'marching': 5446, u'treading': 8782, u'canes': 2294, u'Spectators': 991, u'electricity': 3500, u'sunlit': 8302, u'sports-field': 7981, u'shaving': 7451, u'sinking': 7607, u'helemt': 4576, u'shielding': 7475, u'button-down': 2235, u'yellow': 9447, u'fur': 4120, u'bringing': 2101, u'hula-hoop': 4745, u'equpitment': 3592, u'prize': 6662, u'wooden': 9379, u'satchel': 7250, u'piling': 6388, u'crotch': 2972, u'Desperate': 323, u'sailboarder': 7217, u'Sandy': 900, u'ornamental': 6014, u'glassy': 4223, u'maneuver': 5423, u'snuggles': 7834, u'ornate': 6016, u'tires': 8655, u'elegant': 3503, u'rusty': 7201, u'Hockey': 506, u'turban': 8869, 'END': 353, u'tether': 8552, u'blouse': 1913, u'cooking': 2830, u'sooners': 7884, u'fingers': 3835, u'dangles': 3076, u'brush-land': 2137, u'numeral': 5890, u'pawed': 6239, u'shocks': 7498, 

In [0]:
def new_gen(index1):
  fatch_sample = batch_size
  encoder_input_data = np.zeros((fatch_sample, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
  decoder_input_data = np.zeros((fatch_sample, max_decoder_seq_length, num_decoder_tokens),dtype='float32')
  img_tar_data=np.zeros((fatch_sample, 4096),dtype='float32')
  
  for i in range(fatch_sample):
    
    index = index1 + i
    inp_txt=input_texts[index]
    dec_txt=target_texts[index]
    for t, char in enumerate(inp_txt):
      encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(dec_txt):
      decoder_input_data[i, t, target_token_index[char]] = 1.
        #if t > 0:
        #  decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    img_tar_data[i]=img_mat[index]
  #print("f")
  
  return encoder_input_data , decoder_input_data, img_tar_data
  

In [0]:
import random

def new_gen1(index1):
  fatch_sample = batch_size
  encoder_input_data = np.zeros((fatch_sample, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
  decoder_input_data = np.zeros((fatch_sample, max_decoder_seq_length, num_decoder_tokens),dtype='float32')
  img_tar_data=np.zeros((fatch_sample, 4096),dtype='float32')
  
  for i in range(fatch_sample):
    
    index = random.randint(0,25000)
    inp_txt=input_texts[index]
    dec_txt=target_texts[index]
    for t, char in enumerate(inp_txt):
      encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(dec_txt):
      decoder_input_data[i, t, target_token_index[char]] = 1.
        #if t > 0:
        #  decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    img_tar_data[i]=img_mat[index/5]
  #print("f")
  
  return encoder_input_data , decoder_input_data, img_tar_data
  

In [0]:
# def new_gen_val():
#   fatch_sample = batch_size
#   encoder_input_data = np.zeros((fatch_sample, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
#   decoder_input_data = np.zeros((fatch_sample, max_decoder_seq_length, num_decoder_tokens),dtype='float32')
#   img_tar_data=np.zeros((fatch_sample, 4096),dtype='float32')
  
#   for i in range(fatch_sample):
#     index = np.random.randint(5400,6000)
#     inp_txt=input_texts[index]
#     dec_txt=target_texts[index]
#     for t, char in enumerate(inp_txt):
#       encoder_input_data[i, t, input_token_index[char]] = 1.
#     for t, char in enumerate(dec_txt):
#       decoder_input_data[i, t, target_token_index[char]] = 1.
#         #if t > 0:
#         #  decoder_target_data[i, t - 1, target_token_index[char]] = 1.
#     img_tar_data[i]=img_mat[index]
#   #print("f")
  
#   return encoder_input_data , decoder_input_data, img_tar_data

In [0]:
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

def crop1(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end,:]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name='layer71')


def crop2(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end,:]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name='layer72')

#Lambda(x[1,:,:],name='layer7')

In [0]:
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
#from keras import metrics

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
img_inp_st = Input(shape=(4096,),name='layer00')
#dec_inp_tar = Input(shape=(None, num_encoder_tokens),name='layer000')

encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model([encoder_inputs, decoder_inputs,img_inp_st],[decoder_outputs,inter_mediate_state])
model = Model([encoder_inputs, decoder_inputs,img_inp_st],decoder_outputs)
xent_loss =  K.sum(losses.binary_crossentropy(encoder_inputs, decoder_outputs))
kl_loss = losses.mean_squared_error(img_inp_st, inter_mediate_state)# K.mean(K.square(inter_mediate_state - img_inp_st), axis=-1)
#xent_loss=xent_loss
vae_loss = xent_loss+kl_loss*1000
#vae_loss = xent_loss


#vae_loss = kl_loss*100

model.add_loss(vae_loss)
model.compile(optimizer='rmsprop')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Output "dense_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_2" during training.


In [1]:
state_c

NameError: ignored

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
layer0 (InputLayer)             (None, None, 9486)   0                                            
__________________________________________________________________________________________________
layer3 (LSTM)                   [(None, 512), (None, 20477952    layer0[0][0]                     
__________________________________________________________________________________________________
layer4 (Concatenate)            (None, 1024)         0           layer3[0][1]                     
                                                                 layer3[0][2]                     
__________________________________________________________________________________________________
layer5 (Dense)                  (None, 4096)         4198400     layer4[0][0]                     
__________

In [0]:
for j in range(13,14):
  print ("Super epoch number %d in motion" %j)
  for k in range(0,25):
    index = k*200
    a = new_gen1(index)
    model.fit([a[0],a[1],a[2]],shuffle=True,batch_size=batch_size,epochs=epochs)
  print ("The following six are validation losses:")
  #b = new_gen(6000)
  #model.fit([b[0], b[1],b[2]],shuffle=False,batch_size=batch_size,epochs=1, validation_split=0.99)
  #b = new_gen(6200)
  #model.fit([b[0], b[1],b[2]],shuffle=False,batch_size=batch_size,epochs=1, validation_split=0.99)
  #b = new_gen(6400)
  #model.fit([b[0], b[1],b[2]],shuffle=False,batch_size=batch_size,epochs=1, validation_split=0.99)
  #b = new_gen(6600)
  #model.fit([b[0], b[1],b[2]],shuffle=False,batch_size=batch_size,epochs=1, validation_split=0.99)
  #b = new_gen(6800)
  #model.fit([b[0], b[1],b[2]],shuffle=False,batch_size=batch_size,epochs=1, validation_split=0.99)
  

       
#[inter_mediate_state,
# Run training
#[img_tar_data,
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save model
model.save('s2s1.h5')

Super epoch number 13 in motion
Epoch 1/10
200/200 [==============================] - 6s 29ms/step - loss: 18.5259
Epoch 2/10
200/200 [==============================] - 2s 11ms/step - loss: 16.7708
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 15.9228
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 15.2464
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 13.9512
Epoch 6/10
200/200 [==============================] - 2s 10ms/step - loss: 13.9750
Epoch 7/10
200/200 [==============================] - 2s 10ms/step - loss: 13.5849
Epoch 8/10
200/200 [==============================] - 2s 10ms/step - loss: 12.4900
Epoch 9/10
200/200 [==============================] - 2s 10ms/step - loss: 12.2283
Epoch 10/10
200/200 [==============================] - 2s 10ms/step - loss: 12.0677
Epoch 1/10
200/200 [==============================] - 2s 10ms/step - loss: 18.8904
Epoch 2/10
200/200 [==============================] - 

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'layer71_1/strided_slice:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'layer72_1/strided_slice:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


sample_data


In [0]:
#@title
for j in range(0,5):
  print ("Super epoch number %d in motion" %j)
  for k in range(0,54):
    index = k*100
    a = new_gen(index)
    model.fit([a[0],a[1],a[2]],shuffle=True,batch_size=batch_size,epochs=epochs)
  print ("The following six are validation losses:")
  b = new_gen(5400)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)
  b = new_gen(5500)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)
  b = new_gen(5600)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)
  b = new_gen(5700)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)
  b = new_gen(5800)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)
  b = new_gen(5900)
  model.fit([b[0], b[1],b[2]],shuffle=True,batch_size=batch_size,epochs=1, validation_split=0.99)

In [0]:
model.save('my_new_model.h5')

8thApril (2).ipynb  data  s2s.h5


In [0]:
cp my_new_model.h5 drive/saved_model/

In [0]:
#@title
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
#from keras import metrics

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
img_inp_st = Input(shape=(4096,),name='layer00')
#dec_inp_tar = Input(shape=(None, num_encoder_tokens),name='layer000')

encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model([encoder_inputs, decoder_inputs,img_inp_st],[decoder_outputs,inter_mediate_state])
new_model = Model([encoder_inputs, decoder_inputs,img_inp_st],decoder_outputs)
xent_loss =  K.sum(losses.binary_crossentropy(encoder_inputs, decoder_outputs))
kl_loss = losses.mean_squared_error(img_inp_st, inter_mediate_state)# K.mean(K.square(inter_mediate_state - img_inp_st), axis=-1)
#xent_loss=xent_loss
vae_loss = xent_loss+kl_loss*100
#vae_loss = xent_loss

new_model.add_loss(vae_loss)
new_model.compile(optimizer='rmsprop')




new_model.load_weights('data/my_new_model_plus_100.h5')

In [0]:
model.fit([encoder_input_data, decoder_input_data,img_tar_data],shuffle=True,
          batch_size=batch_size,
          epochs=epochs),
          validation_spl)

In [0]:
cd ..

/content


In [0]:
#@title
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
#from keras import metrics

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
img_inp_st = Input(shape=(4096,),name='layer00')
#dec_inp_tar = Input(shape=(None, num_encoder_tokens),name='layer000')

encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model([encoder_inputs, decoder_inputs,img_inp_st],[decoder_outputs,inter_mediate_state])
model = Model([encoder_inputs, decoder_inputs,img_inp_st],decoder_outputs)
xent_loss =  K.sum(losses.binary_crossentropy(encoder_inputs, decoder_outputs))
kl_loss = losses.mean_squared_error(img_inp_st, inter_mediate_state)# K.mean(K.square(inter_mediate_state - img_inp_st), axis=-1)
#xent_loss=xent_loss
vae_loss = xent_loss+kl_loss*100
#vae_loss = xent_loss

model.add_loss(vae_loss)
model.compile(optimizer='rmsprop')
#[inter_mediate_state,
# Run training
#[img_tar_data,
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data,img_tar_data],shuffle=True,
          batch_size=batch_size,
          epochs=epochs,validation_split=0.2)
# Save model
#model.save('s2s.h5')

In [0]:
model.load_weights('drive/My Drive/alloutput/s2s1.h5')

In [0]:
!config.gpu_options.allow_growth=True

/bin/sh: 1: config.gpu_options.allow_growth=True: not found


In [0]:
!ls

drive  final  sample_data


In [0]:
tr= generator_x()
model.fit_generator(tr,steps_per_epoch=100)

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
  k,l,m = new_gen(5600)
  k.shape

(200, 37, 4600)

In [0]:
#@title
lat1ent_dim=512
def decode_sequence(ind):
    # Encode the input as state vectors.
    #input_seq
    #isis = K.variable(value=input_seq)
    #tmp=K.ran()
    #img_tar_data[ind:ind+1]=img_tar_data[ind:ind+1]-img_tar_data[ind+1:ind+2]+1
    #print(img_tar_data[ind:ind+1])
    #print (img_tar_data)
    get_3rd_layer_output = K.function([new_model.get_layer(name='layer6').input],
                                       [new_model.get_layer(name='layer71').output])
    layer3_output_t1 = get_3rd_layer_output([img_mat[ind:ind+1]])[0]
    get_3rd_layer_output = K.function([new_model.get_layer(name='layer6').input],
                                      [new_model.get_layer(name='layer72').output])
    layer3_output_t2 = get_3rd_layer_output([img_mat[ind:ind+1]])[0]
    states_value=[layer3_output_t1,layer3_output_t2]
    
    #k,l,m = new_gen(5600)
    
    #new_k = k[ind-5600].reshape(1,37,-1)
    #states_value=encoder_model.predict(new_k)
    #states_value=encoder_model.predict(input_seq)
    #print(states_value_d[0])
    #print(states_value[0])
   
      
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['STR']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char+' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'EOS' or sampled_char == '.' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:
lat1ent_dim=512
def decode_sequence(ind):
    # Encode the input as state vectors.
    #input_seq
    #isis = K.variable(value=input_seq)
    #tmp=K.ran()
    #img_tar_data[ind:ind+1]=img_tar_data[ind:ind+1]-img_tar_data[ind+1:ind+2]+1
    #print(img_tar_data[ind:ind+1])
    #print (img_tar_data)
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                       [model.get_layer(name='layer71').output])
    layer3_output_t1 = get_3rd_layer_output([img_mat[ind:ind+1]])[0]
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                      [model.get_layer(name='layer72').output])
    layer3_output_t2 = get_3rd_layer_output([img_mat[ind:ind+1]])[0]
    states_value=[layer3_output_t1,layer3_output_t2]
    
    #k,l,m = new_gen(ind)
    
    #new_k = k[0].reshape(1,37,4600)
    #new_k=np.zeros([1,37,4600])
    #states_value=encoder_model.predict(new_k)
    #states_value=encoder_model.predict(input_seq)
    #print(states_value_d[0])
    #print(states_value[0])
   
      
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['STR']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char+' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'EOS' or sampled_char == '.' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:
lat1ent_dim=512
def decode_sequence(input_seq,ind):
    # Encode the input as state vectors.
    #input_seq
    #isis = K.variable(value=input_seq)
    #tmp=K.ran()
    #img_tar_data[ind:ind+1]=img_tar_data[ind:ind+1]-img_tar_data[ind+1:ind+2]+1
    #print(img_tar_data[ind:ind+1])
    #print (img_tar_data)
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                       [model.get_layer(name='layer71').output])
    layer3_output_t1 = get_3rd_layer_output([img_tar_data[ind:ind+1]])[0]
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                      [model.get_layer(name='layer72').output])
    layer3_output_t2 = get_3rd_layer_output([img_tar_data[ind:ind+1]])[0]
    #states_value=[layer3_output_t1,layer3_output_t2]
    states_value=encoder_model.predict(input_seq)
    #print(states_value_d[0])
    #print(states_value[0])
   
      
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['STR']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char+' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'EOS' or sampled_char == '.' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:
decoded_sentence = decode_sequence(156)
print('-')
print('Input sentence:', input_texts[3690])
print('Decoded sentence:', decoded_sentence)
      #text_file.write(decoded_sentence + '\n')

-
Input sentence: [u'A', u'person', u'doing', u'the', u'backstroke', u'in', u'an', u'outdoor', u'pool', u'.']
Decoded sentence: A man in a red shirt climbs a rock face while the snow . 


In [0]:
text_file = open("8ktst1.txt", "w")


for seq_index in range(0,5000):
    
    # Take one sequence (part of the training set)
    # for trying out decoding.
    #seq_index=10
    
    #input_seq = encoder_input_data[seq_index: seq_index + 1]
    if seq_index%5==0:
      decoded_sentence = decode_sequence(seq_index)
      print('-')
      print('Input sentence:', input_texts[seq_index])
      print('Decoded sentence:', decoded_sentence)
      text_file.write(decoded_sentence + '\n')

text_file.close()

In [0]:
text_file = open("Output1025.txt", "w")


for seq_index in range(25000,6000):
    
    # Take one sequence (part of the training set)
    # for trying out decoding.
    #seq_index=10
    
    #input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(seq_index)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    text_file.write(decoded_sentence + '\n')

text_file.close()

In [0]:
from google.colab import files
files.download('s2s1.h5')

In [0]:
!ls

120output.txt  500output.txt  drive  Oooutput2001.txt  s2s1.h5
130output.txt  70output.txt   final  Oooutput200.txt   sample_data


In [0]:
cp 140output.txt drive/My\ Drive/alloutput/ 

In [0]:
cp s2s1.h5 drive/My\ Drive/havana/ 

In [0]:
cp  drive/My\ Drive/alloutput/s2s1.h5 final/

In [0]:
!ls

drive  final  Oooutput2001.txt	Oooutput200.txt  s2s1.h5  sample_data


In [0]:
thefile = open("Output2.txt", "w")

for item in input_texts:
  item = ' '.join(item)
  thefile.write("%s\n" % item)

thefile.close()